This notebook uses a set of functions and module within the bispy package to interact with the GBIF API to pull back information on WLCI species.

The GBIF records are retrieved and cached based on valid/accepted ITIS species names and the invalid/not accepted ITIS species names.

In [ ]:
#Import needed packages
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed

gbif = bispy.gbif.Gbif()
bis_utils = bispy.bis.Utils()

In [ ]:
# Open up the cached updated ITIS list with valid/accepted species names from ITIS Exploration.ipynb
with open("cache/valid_itis.json", "r") as f:
    valid_itis = json.loads(f.read())

In [ ]:
# Use joblib to run multiple requests for GBIF records in parallel via the valid/accepted ITIS scientific names
gbif_results = Parallel(n_jobs=8)(delayed(gbif.summarize_us_species)(name) for name in [r["scientific_name"] for r in valid_itis])
len(gbif_results)

In [ ]:
# Filter to give just cases where GBIF species name matched with valid/accepted ITIS names
success_gbif = [i for i in gbif_results if i["processing_metadata"]["status"] == "success"]
len(success_gbif)

In [ ]:
#Display records listed as failure for valid/accepted ITIS specie names
fail_gbif = [i for i in gbif_results if i["processing_metadata"]["status"] == "failure"]
fail_gbif

In [ ]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/gbif.json", success_gbif))

In [ ]:
# Open up the cached updated ITIS list with invalid/not accepted species names from ITIS Exploration.ipynb
with open("cache/invalid_itis.json", "r") as f:
    invalid_itis = json.loads(f.read())

In [ ]:
# Use joblib to run multiple requests for GBIF documents in parallel via the invalid/not accepted ITIS scientific names
invalid_result = Parallel(n_jobs=8)(delayed(gbif.summarize_us_species)(name) for name in [r["scientific_name"] for r in invalid_itis]) 

In [ ]:
#Display result to identify the GBIF record that matched the invalid/not accepted ITIS name
invalid_result

In [ ]:
# Cache the retrieved document and return/display a sample for verification
display(bis_utils.doc_cache("cache/invalid_gbif.json", invalid_result))